In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob2
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import math
from numba import cuda
from skimage import exposure
from sklearn.metrics import accuracy_score

# SVM

In [ ]:
class SVM():
  def __init__(self,C=1.0):
    # C error terms
    self.C = C
    self.w = 0
    self.b = 0

  # Hinge Loss Function / Calculation
  def hingeloss(self, w, b, x, y):
    # Regularizer term
    reg = 0.5 * (w * w)

    for i in range(x.shape[0]):
      # Optimization term
      opt_term = y[i] * ((np.dot(w, x[i])) + b)

      # calculating loss
      loss = reg + self.C * max(0, 1-opt_term)
    return loss[0][0]

  def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
    # The number of features in X
    number_of_features = X.shape[1]

    # The number of Samples in X
    number_of_samples = X.shape[0]

    c = self.C

    # Creating ids from 0 to number_of_samples - 1
    ids = np.arange(number_of_samples)

    # Shuffling the samples randomly
    np.random.shuffle(ids)

    # creating an array of zeros
    w = np.zeros((1, number_of_features))
    b = 0
    losses = []

    # Gradient Descent logic
    for i in range(epochs):
      # Calculating the Hinge Loss
      l = self.hingeloss(w, b, X, Y)

      # Appending all losses 
      losses.append(l)
      
      # Starting from 0 to the number of samples with batch_size as interval
      for batch_initial in range(0, number_of_samples, batch_size):
        gradw = 0
        gradb = 0

        for j in range(batch_initial, batch_initial + batch_size):
          if j < number_of_samples:
            x = ids[j]
            ti = Y[x] * (np.dot(w, X[x].T) + b)

            if ti > 1:
              gradw += 0
              gradb += 0
            else:
              # Calculating the gradients
              #w.r.t w 
              gradw += c * Y[x] * X[x]
              # w.r.t b
              gradb += c * Y[x]

        # Updating weights and bias
        w = w - learning_rate * w + learning_rate * gradw
        b = b + learning_rate * gradb

    self.w = w
    self.b = b

    return self.w, self.b, losses

  def predict(self, X):
    prediction = np.dot(X, self.w[0]) + self.b # w.x + b
    return np.sign(prediction)

# HOG

In [ ]:
class HOG:
    def __init__(self, blockSize, cellSize, nbins, sbins, threadsperblock):
        self.blockSize       = blockSize
        self.cellSize        = cellSize
        self.nbins           = nbins
        self.sbins           = sbins
        self.threadsperblock = threadsperblock
    
    @staticmethod
    @cuda.jit
    def __gray_kernel(input, width, height, channel, gray):
        row, col = cuda.grid(2)
        if row >= height or col >= width or channel != 3:
            return
        rgb = input[row][col]
        gray[row][col] = 0.299*rgb[0] + 0.587*rgb[1] + 0.114*rgb[2]
    
    def __gray(self):
        picture = self.picture_array
        # Memory Allocation
        blockspergrid_x = math.ceil(self.picture_array.shape[0] / self.threadsperblock[0])
        blockspergrid_y = math.ceil(self.picture_array.shape[1] / self.threadsperblock[1])
        blockspergrid = (blockspergrid_x, blockspergrid_y)
        gray_dev   = np.empty([self.height, self.width],dtype = float)
        input_dev   = cuda.to_device(self.picture_array)
        gray_device = cuda.device_array_like(gray_dev)
        kernel = self.__gray_kernel
        kernel[blockspergrid, self.threadsperblock](input_dev, self.width, self.height, self.channel, gray_device)
        gray_dev = gray_device.copy_to_host()
        self.gray = gray_dev
    
    @staticmethod
    @cuda.jit
    def __calc_gradient_kernel(input, width, height, output_x, output_y):
      row, col = cuda.grid(2)
      if (row>=height) or (col>=width):
        return
      for i in range(-1,2):
        pixel_r = row + i
        pixel_r = min(max(0, pixel_r), height - 1)
        output_y[row, col] += input[pixel_r,col] * i

        pixel_c = col + i
        pixel_c = min(max(0, pixel_c), width - 1)
        output_x[row,col] += input[row,pixel_c] * i
        
    def __calc_gradient(self):
        blockspergrid_x = math.ceil(self.picture_array.shape[0] / self.threadsperblock[0])
        blockspergrid_y = math.ceil(self.picture_array.shape[1] / self.threadsperblock[1])
        blockspergrid = (blockspergrid_x, blockspergrid_y)
        gradient_x_dev = np.zeros((self.height, self.width))
        gradient_y_dev = np.zeros((self.height, self.width))

        gradient_x_device = cuda.to_device(gradient_x_dev)
        gradient_y_device = cuda.to_device(gradient_y_dev)
        gray_device       = cuda.to_device(self.gray)
        self.__calc_gradient_kernel[blockspergrid, self.threadsperblock]\
                (gray_device, self.width, self.height, gradient_x_device, gradient_y_device)

        gradient_x_dev = gradient_x_device.copy_to_host()
        gradient_y_dev = gradient_y_device.copy_to_host()
        return gradient_x_dev, gradient_y_dev
    
    def __calc_direc_mag(self):
        self.__gray()
        gradient_x, gradient_y = self.__calc_gradient()
        self.magnitude = np.sqrt(np.square(gradient_x)+np.square(gradient_y))
        self.direction = np.mod(np.add(360, np.rad2deg(np.arctan2(np.array(gradient_y), np.array(gradient_x)))), 360)
    
    @staticmethod
    @cuda.jit
    def __hist_kernel(direction, magnitude, width, height, sbin, cell_size, result_out):    
        cur_r, cur_c  = cuda.grid(2)

        idy   = int(cur_r//cell_size[0])
        idx   = int(cur_c//cell_size[1])
        # kiểm tra
        if cur_r>=height or cur_c>= width:
            return 
        thread_direction = direction[cur_r][cur_c]
        thread_mag       = magnitude[cur_r][cur_c]
        # chia lấy phần nguyên và phần dư
        quotient  = int(thread_direction//sbin)
        remainder =     thread_direction % sbin

        if remainder==0:
            cuda.atomic.add(result_out, (idy, idx, quotient), thread_mag)
        else:
            first_bin = quotient

            second_bin   = first_bin+1
            need_to_add    = thread_mag*((second_bin*sbin - thread_direction)/(second_bin*sbin - first_bin*sbin))
            cuda.atomic.add(result_out, (idy, idx, first_bin), need_to_add)

            second_bin_idx = second_bin
            if second_bin > 8:
                second_bin_idx = 0
            need_to_add_2  = thread_mag*((thread_direction - first_bin*sbin)/(second_bin*sbin - first_bin*sbin))   
            cuda.atomic.add(result_out, (idy, idx, second_bin_idx), need_to_add_2)
    
    def __all_hist(self):
        blockspergrid_x = math.ceil(self.picture_array.shape[0] / self.threadsperblock[0])
        blockspergrid_y = math.ceil(self.picture_array.shape[1] / self.threadsperblock[1])
        blockspergrid = (blockspergrid_x, blockspergrid_y)
        hist_dev    = np.empty([self.n_cell[0], self.n_cell[1], self.nbins],dtype = np.float64)
        d_direction = cuda.to_device(self.direction)
        d_magnitude = cuda.to_device(self.magnitude)
        d_cell_size = cuda.to_device(self.cellSize)
        hist_device = cuda.device_array_like(hist_dev)
        kernel = self.__hist_kernel
        kernel[blockspergrid, self.threadsperblock]\
                            (d_direction, d_magnitude, self.width, self.height, self.sbins, d_cell_size, hist_device)
        self.hist = hist_device.copy_to_host()
        
        
    def compute_HOG(self, picture):
        self.picture_array = picture
        self.height, self.width, self.channel = self.picture_array.shape
        self.n_cell  = (self.height//self.cellSize[0], self.width//self.cellSize[1])
        self.n_block = (self.n_cell[0] - self.blockSize[0] + 1, self.n_cell[1] - self.blockSize[1] + 1)
        
        self.__calc_direc_mag()
        self.__all_hist()
        norm_array_size = self.n_block[0] * self.n_block[1] * self.blockSize[0] * self.blockSize[1] * self.nbins
        l2 = np.empty(self.n_block)
        for i in range(self.n_block[0]):
            for j in range(self.n_block[1]):
                l2[i][j] = math.sqrt(np.sum(np.square(self.hist[i:i+2, j:j+2])))
        norm_block = np.zeros((self.n_block[0], self.n_block[1], self.blockSize[0], self.blockSize[1], self.nbins))
        for y in range(self.n_block[0]):
            for x in range(self.n_block[1]):
                out = self.hist[y: y + self.blockSize[0], x: x + self.blockSize[1]] / (l2[y][x] + 1)
                norm_block[y][x] = out
        self.HOG        = norm_block.flatten()
        self.norm_block = norm_block
        return self.HOG

# Đọc data

In [ ]:
cell_size = (8,8)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
list_cat = glob2.glob('drive/MyDrive/ltssud/data/Cat/**')
print(len(list_cat))
images = []
labels = []

for i in range(100):
    img = Image.open(list_cat[i])
    img = img.resize((400,400), Image.LANCZOS)
    images.append(np.array(img))
    labels.append(1)

100


In [ ]:
list_dog = glob2.glob('drive/MyDrive/ltssud/data/Dog/**')

for i in range(100):
    img = Image.open(list_dog[i])
    img = img.resize((400,400), Image.LANCZOS)
    images.append(np.array(img))
    labels.append(-1)

In [ ]:
X = np.array(images)
y = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Thử nghiệm SVM không có HOG

In [ ]:
x_train_nohog = np.array([x.flatten() for x in X_train])
x_test_nohog = np.array([x.flatten() for x in X_test])

In [ ]:
svm = SVM()
svm.fit(x_train_nohog, y_train)
pred = svm.predict(x_test_nohog)
accuracy_score(pred, y_test)

0.43333333333333335

# SVM không HOG của sklearn

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
y_train_sklearn = np.array([str(y) for y in y_train])
y_test_sklearn = np.array([str(y) for y in y_test])

In [ ]:
svm = SVC(kernel='linear')
svm.fit(x_train_nohog, y_train_sklearn)
pred = svm.predict(x_test_nohog)
accuracy_score(pred, y_test_sklearn)

0.4666666666666667

# SVM có HOG

In [ ]:
def big_fit(X_train, y_train, hog, svm):
  x_train = []
  t = 0
  for x in X_train:
    print(t, end="  ")
    x_train.append(hog.compute_HOG(x))
    t+=1
  x_train = np.array(x_train)
  svm.fit(x_train, y_train)

def big_predict(X_test, y_test, hog, svm):
  x_test = []
  t = 0
  for x in X_test:
    print(t, end = "  ")
    x_test.append(hog.compute_HOG(x))
    t+=1
  x_test = np.array(x_test)
  return svm.predict(x_test)

In [ ]:
hog = HOG((2,2), (8,8), 9, 40,(32,32))
svm = SVM()

In [ ]:
big_fit(X_train, y_train, hog, svm)

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  

In [ ]:
test_predict = big_predict(X_test, y_test, hog, svm)

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  

In [ ]:
accuracy_score(y_test, test_predict)

0.6